In [ ]:
from __future__ import division, print_function
import sys
if '..' not in sys.path:
    sys.path.append('..')

import theano
import theano.tensor as T
import lasagne

import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.metrics import confusion_matrix

In [ ]:
%matplotlib inline

# Load datasets

- the datasets are loaded/built.
- The batchsize is defined
- half of the data name (the source part) is defined

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


## Circles

In [ ]:
from datasets.toys import make_circles
from datasets.utils import make_dataset

n_samples = 300  # Number of sample per class
n_classes = 5
n_dim = 2
batchsize = 80
_data_name = 'Circles'
X, y = make_circles(n_samples=n_samples, n_classes=n_classes, n_dim=n_dim)
source_data = make_dataset(X, y, batchsize=batchsize)

## Clouds

In [ ]:
from datasets.toys import make_clouds
from datasets.utils import make_dataset

n_samples = 300  # Number of sample per class
n_classes = 8
batchsize = 80
_data_name = 'Clouds'
X, y = make_clouds(n_samples=n_samples, n_classes=n_classes)
source_data = make_dataset(X, y, batchsize=batchsize)

## X

In [ ]:
from datasets.toys import make_X
from datasets.utils import make_dataset

n_samples = 300  # Number of sample per class
n_classes = 5
batchsize = 80
_data_name = 'X'
X, y = make_X(n_samples=n_samples, n_classes=n_classes)
source_data = make_dataset(X, y, batchsize=batchsize)

## Moons

In [ ]:
from datasets.toys import make_moons
from datasets.utils import make_dataset

n_samples = 500
batchsize = 80
_data_name = 'Moons'
X, y = make_moons(n_samples=n_samples)
source_data = make_dataset(X, y, batchsize=batchsize)

## MNIST

In [ ]:
from datasets.mnist import load_mnist
from datasets.utils import make_dataset

batchsize = 500
_data_name = 'MNIST'
X, y = load_mnist()
source_data = make_dataset(X, y, batchsize=batchsize)

# Transform datasets

- the transformed datasets are built.
- last part of the data name (the target part) is defined

- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


### Data rotated

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_Rotated'
angle = 35

X_t, y_t = transform.rotate(X, y)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

### Data . Random Matrix

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_RMat'

X_t, y_t = transform.random_mat(X, y, normalize=False)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

### Data . Diag Dominant matrix

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_Diag'

X_t, y_t = transform.diag_dominant(X, y, normalize=True)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

### Data Mirror

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_Mirror'

X_t, y_t = transform.mirror(X, y)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

### Data Random Permutation

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_Rperm'

X_t, y_t = transform.random_permut(X, y)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

### Grid Bend

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_GridBend'
nx = 3
ny = 3
grid_noise = 0.3

X_t, y_t, grid = transform.grid_bend(X, y, nx=nx, ny=ny, noise=grid_noise)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

### Apply function

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

data_name = _data_name+'_Apply'

def _fun(X):
    f = np.vectorize((lambda x: x if x<0 else x-x*x))
    return f(X)
#     return X-X**2
#     return 2*X
X_t = _fun(X)
y_t = np.copy(y)
target_data = make_dataset(X_t, y_t, batchsize=batchsize)
domain_data = make_domain_dataset([source_data, target_data])
corrector_data = make_corrector_dataset(source_data, target_data)

In [ ]:
import visual
fig, ax = visual.source_2D(X, y)
fig, ax = visual.target_2D(X_t, y_t, ax=ax)
ax.plot(grid.xxx, grid.yyy, '--');
ax.plot(grid.xxx.T, grid.yyy.T, '--');

# Build the Neural Network


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


## Network building
Start with the variables

In [ ]:
from nn.rgl import ReverseGradientLayer
from nn.compilers import crossentropy_sgd_mom, squared_error_sgd_mom, adversarial
from nn.training import Trainner, training


## Parameters

- Learning rates
- Hyper params
- Logger

In [ ]:
from logs import log_fname, new_logger, empty_logger

hp_lambda = 0.

label_rate = 0.1
label_mom = 0.9

domain_rate = 0.1
domain_mom = 0.9

# Get a logger
logger = new_logger()


### Theano variables

In [ ]:
_shape = np.shape(source_data['X_train'])
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

# Prepare Theano variables for inputs and targets
if n_dim == 2:
    input_var = T.matrix('inputs')
    src_var = T.matrix('src')
    target_var = T.matrix('targets')
    shape = (batchsize,) + _shape[1:]
elif n_dim == 3:
    input_var = T.tensor3('inputs')
    src_var = T.tensor3('src')
    target_var = T.tensor3('targets')
    shape = (batchsize,) + _shape[1:]
elif n_dim == 4:
    input_var = T.tensor4('inputs')
    src_var = T.tensor4('src')
    target_var = T.tensor4('targets')
    shape = (batchsize,) + _shape[1:]

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


## Architecture


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


In [ ]:
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape, input_var=input_var)
src_layer = lasagne.layers.InputLayer(shape=shape, input_var=src_var)
dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=5,
                nonlinearity=lasagne.nonlinearities.tanh,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
# dropout_1 = lasagne.layers.DropoutLayer(dense_1, p=0.5)
dense_2 = lasagne.layers.DenseLayer(
                dense_1,
                num_units=5,
                nonlinearity=lasagne.nonlinearities.tanh,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
feature = lasagne.layers.DenseLayer(
                input_layer,
                num_units=np.prod(shape[1:]),  # Should have same number as the input dimension
                nonlinearity=None,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
reshaper = lasagne.layers.ReshapeLayer(feature, (-1,) + shape[1:])
output_layer = reshaper

# Logs
logger.info('Building the neural network architecture for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


## Compiling 


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


In [ ]:
# Logs
logger.info('Compiling the neural network for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Compilation
corrector_trainner = Trainner(squared_error_sgd_mom(output_layer, lr=label_rate, mom=0, 
                                                    target_var=target_var,
                                                   regularization=None, reg_param=0.1), 
                             'corrector',)

domain_trainner = Trainner(adversarial([src_layer, output_layer], hp_lambda=hp_lambda,
                                      lr=domain_rate, mom=domain_mom),
                           'domain')


## Add preprocessing for alignment


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


In [ ]:
from align_learn.preprocess import classwise_shuffle, exhaustive_clostest, cluster_preprocess, build_clusters

# Choose preprocessing :
# corrector_trainner.preprocess = classwise_shuffle
# corrector_trainner.preprocess = exhaustive_clostest
corrector_trainner.preprocess = cluster_preprocess

model_name = ''
if corrector_trainner.preprocess is classwise_shuffle:
    model_name = 'Classwise_Corrector'
    corrector_data['labels'] = source_data['y_train']
elif corrector_trainner.preprocess is exhaustive_clostest:
    model_name = 'K-closest_Corrector'
    corrector_data['labels'] = source_data['y_train']
elif corrector_trainner.preprocess is cluster_preprocess:
    model_name = 'Cluster_Corrector'
    n_clusters = 6
    corrector_data['k'] = -1
    y = source_data['y_train']
    classes = np.unique(y)

    # Build the clusters for target data
    centers_array, clusters_label, centers_labels = build_clusters(corrector_data['X_train'],
                                                                   y, n_clusters=n_clusters)
    corrector_data['X_train_centers'] = centers_array
    corrector_data['X_train_clusters'] = clusters_label
    corrector_data['centers_labels'] = centers_labels
    
    # Build the clusters for source data
    centers_array, clusters_label, centers_labels = build_clusters(corrector_data['y_train'],
                                                                   y, n_clusters=n_clusters)
    corrector_data['y_train_centers'] = centers_array
    corrector_data['y_train_clusters'] = clusters_label

else:
    model_name = 'Pairwise_Corrector'


# Train the neural network


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


Reset the counter and the stats

In [ ]:
logger.warn('Reset the epoch counter and saved statistics')
epoch_counter = 0
final_stats = {}


## Training loop 

In [ ]:
def do_n_epoch(n_epoch):
    global epoch_counter, logger, final_stats, hp_lambda
    global corrector_data, domain_data, corrector_trainner, domain_trainner
    epoch_counter += n_epoch
    trainers = [corrector_trainner,]
    datas = [corrector_data,]
    #  If hp_lambda == 0 no need to train adversarial (faster computation)
    if hp_lambda != 0.0:
        trainers.append(domain_trainner)
        datas.append(domain_data)
    # Now do the trainning part !
    logger.info('Trainning the neural network for {} additional epochs ({} total)'.format(n_epoch, epoch_counter))
    stats = training(trainers, datas, num_epochs=n_epoch, logger=None)
    final_stats = {k: (final_stats[k]+v if k in final_stats else v) for k, v in stats.items()}

# Plot results


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


## 2D Data plot

In [ ]:
import visual

def my_2D_plot(source_data, target_data, corrector_data, trainer):
    """
    Plot things
    """
    # Compute the correction on test data
    corrected_data = {
        'X_test': np.array(corrector_trainner.output(corrector_data['X_test'])).reshape((-1, 2)),
    }
    # Init figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
    
    # Plot data test points (source + corrected) on left fig
    visual.source_2D(source_data['X_test'], source_data['y_test'], ax=ax1);
    visual.corrected_2D(corrected_data['X_test'], source_data['y_test'], ax=ax1);

    # Plot data test points (source + target) on right fig
    visual.source_2D(source_data['X_test'], source_data['y_test'], ax=ax2);
    visual.target_2D(target_data['X_test'], target_data['y_test'], ax=ax2);
    
    # Plot cluster centers and cluster mapping 
#     if 'preprocess' in corrector_data and 'X_train_centers' in corrector_data:
#         corrected_data['X_train_centers'] = np.array(
#             corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
#         idx = corrector_data['preprocess']
#         X = np.array(corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
#         Y = corrector_data['y_train_centers']
#         Y = Y[idx]
#         visual.centers_source(Y, ax=ax1)
#         visual.centers_corrected(X, ax=ax1)
#         visual.mapping(X, Y, ax=ax1)
#         visual.centers_source(Y, ax=ax2)
#         visual.centers_target(corrector_data['X_train_centers'], ax=ax2)
    return fig, [ax1, ax2]

In [ ]:

def grid_view(X, y, trainer):
    """
    Plot the corrected grid
    """
    nx = ny = 20
    x_min, x_max = np.min(X[:, 0])*2, np.max(X[:, 0])*2
    y_min, y_max = np.min(X[:, 1])*2, np.max(X[:, 1])*2
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, num=nx),
                         np.linspace(y_min, y_max, num=ny))
    X_grid = np.vstack([xx.ravel(), yy.ravel()]).T
    y_grid = np.hstack([np.ones(nx)*i for i in range(ny)])
    
#     Transform:
    X_tgt, y_tgt = transform.rotate(X_grid, y_grid, angle=angle)
    
    X_corr = np.array(trainer.output(X_tgt)).reshape((-1, 2))
    xx = X_corr[:, 0].reshape(ny, nx)
    yy = X_corr[:, 1].reshape(ny, nx)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    # Plot data test points (source + corrected) on left fig
    visual.corrected_2D(X, y, ax=ax1)
    ax1.plot(xx, yy)
    ax1.plot(xx.T, yy.T)

    xx = X_tgt[:, 0].reshape(ny, nx)
    yy = X_tgt[:, 1].reshape(ny, nx)
    visual.corrected_2D(X, y, ax=ax2)
    ax2.plot(xx, yy)
    ax2.plot(xx.T, yy.T)
    return fig, (ax1, ax2)
    

In [ ]:
fig_dir = '/home/victor/Workspace/Stage/DANN/fig/'
fig_title = fig_dir+data_name+model_name
from datasets.utils import Dataset

# Play !


- [1. Loading of datasets](#Load-datasets)
- [2. Transformation of datasets](#Transform-datasets)
- [3. Init of the NN](#Build-the-Neural-Network)
- [4. Architecture](#Architecture)
- [5. Compiling](#Compiling)
- [6. Preprocessing for alignment](#Add-preprocessing-for-alignment)
- [7. Playground](#Play-!)


In [ ]:
print(model_name, data_name)
do_n_epoch(0)

# ================
# Compute the correction on test data
# ================
corrected_data = Dataset(
    X_test=np.array(corrector_trainner.output(corrector_data['X_test'])).reshape((-1, 2)),
    y_test=source_data.y_test
)


# ================
# Data visualisation
# ================
if not n_dim > 2:
    # Init figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

    # Plot data test points (source + corrected) on left fig
    visual.source_2D(source_data.X_test, source_data.y_test, ax=ax1);
    visual.corrected_2D(corrected_data.X_test, source_data.y_test, ax=ax1);
    visual.add_legend(ax1, title=data_name+model_name)

    # Plot data test points (source + target) on right fig
    visual.source_2D(source_data.X_test, source_data.y_test, ax=ax2);
    visual.target_2D(target_data.X_test, target_data.y_test, ax=ax2);
    visual.add_legend(ax2, title=data_name+model_name)
    # Plot cluster centers and cluster mapping 
#     if 'preprocess' in corrector_data and 'X_train_centers' in corrector_data:
#         corrected_data['X_train_centers'] = np.array(
#             corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
#         idx = corrector_data['preprocess']
#         X = np.array(corrector_trainner.output(corrector_data['X_train_centers'])).reshape((-1, 2))
#         Y = corrector_data['y_train_centers']
#         Y = Y[idx]
#         visual.centers_source(Y, ax=ax1)
#         visual.centers_corrected(X, ax=ax1)
#         visual.mapping(X, Y, ax=ax1)
#         visual.centers_source(Y, ax=ax2)
#         visual.centers_target(corrector_data['X_train_centers'], ax=ax2)
#     SAVE
    fig.savefig(fig_title+'-DATA.png')
    fig.show()

# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(final_stats, regex='corrector .* loss')
#     SAVE
fig.tight_layout()
fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
visual.learning_curve(final_stats, regex='domain.* acc');

# ================
# Grid check
# ================
fig, (ax1, ax2) = grid_view(corrected_data.X_test, corrected_data.y_test, corrector_trainner)
fig.savefig(fig_title+'-GridCheck.png')

# ================
# Image samples
# ================
if data_name.startswith('MNIST'):
    visual.img_samples([source_data, target_data, 
                     {'X_test': np.array(corrector_trainner.output(
                    corrector_data['X_test'])).reshape((-1,)+corrector_data['X_test'].shape[1:])}])

# ================
# Weights visualisation
# ================
layers = lasagne.layers.get_all_layers(feature)
fig, axes = plt.subplots(len(layers)//2, 2, figsize=(20, 6))
axes = axes.ravel()
for i, l in enumerate(layers[1:]):
    if hasattr(l, 'W'):
        visual.mat(l.W.get_value(), ax = axes[i])
        visual.add_legend(axes[i], title='Layer {} Weights'.format(i))
#     SAVE
fig.savefig(fig_title+'-W.png')

# ================
# Sanity check
# ================
print('Loss : Identitée')
print(np.mean((source_data['X_test']-target_data['X_test'])**2))

In [ ]:
'Done'

### TSNE